<a href="https://colab.research.google.com/github/oceanhackweek/Oceans19-data-science-tutorial/blob/master/notebooks/5_ocean_ssh_example_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sea Surface Altimetry Data Analysis

<img src="http://marine.copernicus.eu/documents/IMG/SEALEVEL_GLO_SLA_MAP_L4_REP_OBSERVATIONS_008_027.png" 
     width="15%" 
     align=left
     alt="Globe">

For this example we will use gridded [sea-surface altimetry data from The Copernicus Marine Environment](http://marine.copernicus.eu/services-portfolio/access-to-products/?option=com_csw&view=details&product_id=SEALEVEL_GLO_PHY_L4_REP_OBSERVATIONS_008_047):

This is a widely used dataset in physical oceanography and climate.

The dataset has already been extracted from copernicus and stored in google cloud storage in [xarray-zarr](http://xarray.pydata.org/en/latest/io.html#zarr) format.

In [0]:
%matplotlib inline

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import dask.array as dsa

plt.rcParams['figure.figsize'] = (15,10)

### Initialize Dataset

Here we load the dataset from the zarr store. Note that this very large dataset initializes nearly instantly, and we can see the full list of variables and coordinates.

In [0]:
!pip install intake
!pip install intake-xarray
!pip install gcsfs

In [0]:
import intake
cat = intake.Catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/ocean.yaml")
ds = cat["sea_surface_height"].to_dask()
ds

### Examine Metadata

For those unfamiliar with this dataset, the variable metadata is very helpful for understanding what the variables actually represent

In [0]:
for v in ds.data_vars:
    print('{:>10}: {}'.format(v, ds[v].attrs['long_name']))

## Visually Examine Some of the Data

Let's do a sanity check that the data looks reasonable:

In [0]:
plt.rcParams['figure.figsize'] = (15, 8)
ds.sla.sel(time='2000-01-01', method='nearest').plot()

### Same thing using interactive graphics

In [0]:
!pip install bokeh==1.3.4
!pip install datashader
!pip install -q holoviews
!pip install dask

In [0]:
import holoviews as hv
from holoviews.operation.datashader import regrid
hv.extension('bokeh')

In [0]:
import os
os.environ['HV_DOC_HTML'] = 'true'
#%env HV_DOC_HTML=true

import numpy as np
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

In [0]:
hv.extension('bokeh')
dataset = hv.Dataset(ds.sla)#.sel(time=slice('2000-01-01', '2001-01-02')))
hv_im = (dataset.to(hv.Image, ['longitude', 'latitude'], dynamic=True)
                #.redim.range(sla=(-0.5, 0.5))
                .options(cmap='RdBu_r', width=800, height=450, colorbar=True))

#%output holomap='scrubber' fps=2
regrid(hv_im, precompute=True)

### Create and Connect to Dask Distributed Cluster

In [0]:
from dask.distributed import Client, progress

** ☝️ Don't forget to click the link above to view the scheduler dashboard! **

In [0]:
client = Client()
client

## Timeseries of Global Mean Sea Level

Here we make a simple yet fundamental calculation: the rate of increase of global mean sea level over the observational period.

In [0]:
# the number of GB involved in the reduction
ds.sla.nbytes/1e9

In [0]:
# the computationally intensive step
sla_timeseries = ds.sla.mean(dim=('latitude', 'longitude')).load()

In [0]:
sla_timeseries.plot(label='full data')
sla_timeseries.rolling(time=365, center=True).mean().plot(label='rolling annual mean')
plt.ylabel('Sea Level Anomaly [m]')
plt.title('Global Mean Sea Level')
plt.legend()
plt.grid()

In order to understand how the sea level rise is distributed in latitude, we can make a sort of [Hovmöller diagram](https://en.wikipedia.org/wiki/Hovm%C3%B6ller_diagram).

In [0]:
sla_hov = ds.sla.mean(dim='longitude').load()

In [0]:
fig, ax = plt.subplots(figsize=(12, 4))
sla_hov.name = 'Sea Level Anomaly [m]'
sla_hov.transpose().plot(vmax=0.2, ax=ax)

We can see that most sea level rise is actually in the Southern Hemisphere.

## Sea Level Variability

We can examine the natural variability in sea level by looking at its standard deviation in time.

In [0]:
sla_std = ds.sla.std(dim='time').load()
sla_std.name = 'Sea Level Variability [m]'

In [0]:
ax = sla_std.plot()
_ = plt.title('Sea Level Variability')